<a href="https://colab.research.google.com/github/Azariagmt/Twitter-Data-Analysis/blob/notebooks/notebooks/Data_Cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Second round of cleaning in notebook env

In [17]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [18]:
df = pd.read_csv('/content/processed_tweet_data.csv')

In [19]:
df.head()

,id,created_at,source,polarity,subjectivity,original_text,lang,favorite_count,retweet_count,original_author,followers_count,friends_count,possibly_sensitive,hashtags,user_mentions,location
0,0,Fri Jun 18 17:55:49 +0000 2021,"<a href=""http://twitter.com/download/iphone"" r...",0.166667,0.188889,"🚨Africa is ""in the midst of a full-blown third...",en,548.0,612.0,ketuesriche,551,351,NaN,[],"[{'screen_name': 'WHOAFRO', 'name': 'WHO Afric...",Mass
1,1,Fri Jun 18 17:55:59 +0000 2021,"<a href=""https://mobile.twitter.com"" rel=""nofo...",0.133333,0.455556,"Dr Moeti is head of WHO in Africa, and one of ...",en,195.0,92.0,Grid1949,66,92,NaN,[],[],"Edinburgh, Scotland"
2,2,Fri Jun 18 17:56:07 +0000 2021,"<a href=""http://twitter.com/download/iphone"" r...",0.316667,0.483333,Thank you @research2note for creating this ama...,en,2.0,1.0,LeeTomlinson8,1195,1176,NaN,"[{'text': 'red4research', 'indices': [87, 100]}]","[{'screen_name': 'Research2note', 'name': 'Res...",NaN
3,3,Fri Jun 18 17:56:10 +0000 2021,"<a href=""https://mobile.twitter.com"" rel=""nofo...",0.086111,0.197222,"Former Pfizer VP and Virologist, Dr. Michael Y...",en,1580.0,899.0,RIPNY08,2666,2704,NaN,"[{'text': 'Covid19', 'indices': [145, 153]}]",[],NaN
4,4,Fri Jun 18 17:56:20 +0000 2021,"<a href=""http://twitter.com/download/android"" ...",0.280000,0.620000,I think it’s important that we don’t sell COVA...,en,72.0,20.0,pash22,28250,30819,NaN,[],"[{'screen_name': 'TexasChildrens', 'name': ""Te...",United Kingdom


In [20]:
# Form a new data frame (named cleanTweet), containing columns  clean-text  and  polarity .
cleanTweet = df[['polarity', 'original_text']]
cleanTweet.head()

,polarity,original_text
0,0.166667,"🚨Africa is ""in the midst of a full-blown third..."
1,0.133333,"Dr Moeti is head of WHO in Africa, and one of ..."
2,0.316667,Thank you @research2note for creating this ama...
3,0.086111,"Former Pfizer VP and Virologist, Dr. Michael Y..."
4,0.280000,I think it’s important that we don’t sell COVA...


### Basic preprocessing


In [21]:
cleanTweet.dtypes

polarity         float64
original_text     object
dtype: object

## important steps in preprocessing text
Make text all lower case <br>
Remove punctuation <br>
Remove numerical values <br>
Remove common non-sensical text <br>
Tokenize text <br>
Remove stop words <br>

In [22]:
# nltk package contains stop words we'll remove using regular expressions
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop = stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [23]:
# convert all original_text fields to strings
cleanTweet['original_text'] = cleanTweet['original_text'].astype(str)
cleanTweet['clean_text'] = cleanTweet['original_text'].str.replace('[^\w\s]','')
cleanTweet['clean_text'] = cleanTweet['clean_text'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
cleanTweet['clean_text'] = cleanTweet['clean_text'].apply(lambda x: " ".join(x.lower() for x in x.split()))
# re.sub('\w*\d\w*', '', text)
cleanTweet['clean_text'] = cleanTweet['clean_text'].str.replace('\w*\d\w*','')
cleanTweet.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

,polarity,original_text,clean_text
0,0.166667,"🚨Africa is ""in the midst of a full-blown third...",africa midst fullblown third wave coronavirus ...
1,0.133333,"Dr Moeti is head of WHO in Africa, and one of ...",dr moeti head who africa one best public healt...
2,0.316667,Thank you @research2note for creating this ama...,thank creating amazing campaign amp turning s...
3,0.086111,"Former Pfizer VP and Virologist, Dr. Michael Y...",former pfizer vp virologist dr michael yeadon ...
4,0.280000,I think it’s important that we don’t sell COVA...,i think important dont sell covax short it sti...


In [24]:
# normally we tokenize using words so each word as a token
#Lines 4 to 6
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()
cleanTweet['clean_text'] = cleanTweet['clean_text'].apply(lambda x: " ".join([stemmer.stem(word) for word in x.split()]))

cleanTweet[['polarity','original_text','clean_text']].head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,polarity,original_text,clean_text
0,0.166667,"🚨Africa is ""in the midst of a full-blown third...",africa midst fullblown third wave coronaviru h...
1,0.133333,"Dr Moeti is head of WHO in Africa, and one of ...",dr moeti head who africa one best public healt...
2,0.316667,Thank you @research2note for creating this ama...,thank creat amaz campaign amp turn social medi...
3,0.086111,"Former Pfizer VP and Virologist, Dr. Michael Y...",former pfizer vp virologist dr michael yeadon ...
4,0.280000,I think it’s important that we don’t sell COVA...,i think import dont sell covax short it still ...


In [25]:
# Write a function text_category that takes a value p and returns, depending on the value of p, a string 'positive', 'negative' or 'neutral'.
def text_category(p):
  if p > 0:
    return 'positive'
  elif p < 0:
    return 'negative'
  else: return 'neutral'

In [26]:
# Apply this function (text_category) on the  polarity  column of cleanTweet in 1 above to form a new column called  score  in cleanTweet.

scores = pd.Series([text_category(row_val) for row_val in cleanTweet['polarity']])
cleanTweet = pd.concat([cleanTweet, scores.rename("score")], axis=1)
cleanTweet.head()

,polarity,original_text,clean_text,score
0,0.166667,"🚨Africa is ""in the midst of a full-blown third...",africa midst fullblown third wave coronaviru h...,positive
1,0.133333,"Dr Moeti is head of WHO in Africa, and one of ...",dr moeti head who africa one best public healt...,positive
2,0.316667,Thank you @research2note for creating this ama...,thank creat amaz campaign amp turn social medi...,positive
3,0.086111,"Former Pfizer VP and Virologist, Dr. Michael Y...",former pfizer vp virologist dr michael yeadon ...,positive
4,0.280000,I think it’s important that we don’t sell COVA...,i think import dont sell covax short it still ...,positive


In [27]:
# remove nan tweets
cleanTweet = cleanTweet[cleanTweet['clean_text']!="nan"]

## Document term matrix
For many of the techniques we'll be using in future notebooks, the text must be tokenized, meaning broken down into smaller pieces. The most common tokenization technique is to break down text into words. We can do this using scikit-learn's CountVectorizer, where every row will represent a different document and every column will represent a different word.

In [28]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer((3,3), stop_words="english")
# count vectorizer to remove stopwords

In [29]:
# X_train_counts = count_vect.fit_transform(X_train)
data_cv = count_vect.fit_transform(cleanTweet['clean_text'])
data_dtm = pd.DataFrame(data_cv.toarray(), columns=count_vect.get_feature_names())
data_dtm.index = cleanTweet.index
data_dtm

,_africanunion,_mamadeb,aarc_tweet,abdala,abil,abl,ableg,abolish,abpoli,abroad,abt,academi,acbwodah,acceler,accept,access,accomplish,accord,accumul,accur,achiev,acip,acknowledg,acquir,acr,actacceler,action,activ,activist,actogeth,actual,acut,ad,adapt,add,addit,address,adelaid,adenoviru,adequ,...,write,writer,wrong,wrote,wsinovac,wth,wud,wuhan,wuhanlableak,xavierbecerra,xingp,yale,yard,ye,yeadon,yeah,year,yeg,yesterday,yield,yochayz,york,youll,young,younger,youtub,youv,ystrday,yunu,yyc,zambia,zero,zika,zimbabw,zip,zyklonb,กระทรวงพฒนาสงคม,ขาวดวน,คนไรบาน,วคซน
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3715,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3716,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3717,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
3718,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


## Pickle for later use

In [30]:
import pickle
# Let's pickle it for later use
data_dtm.to_pickle("dtm.pkl")
# Let's also pickle the cleaned data (before we put it in document-term matrix format) and the CountVectorizer object
cleanTweet.to_pickle('data_clean.pkl')
pickle.dump(count_vect, open("count_vect.pkl", "wb"))

## References
https://www.pluralsight.com/guides/building-features-from-text-data <br>
https://github.com/adashofdata/nlp-in-python-tutorial/blob/master/1-Data-Cleaning.ipynb
